In [14]:
%matplotlib inline

In [15]:
from hashlib import md5, sha256, sha512
from time import time
from string import printable
from itertools import product, count
from csv import writer

## Break MD5 hashing algorithm using brute force and collision

MD5: The fastest and shortest generated hash (16 bytes). The probability of just two hashes accidentally colliding is approximately: 1.47*10-29.

SHA1: Is generally 20% slower than md5, the generated hash is a bit longer than MD5 (20 bytes). The probability of just two hashes accidentally colliding is approximately: 1*10-45

SHA256: The slowest, usually 60% slower than md5, and the longest generated hash (32 bytes). The probability of just two hashes accidentally colliding is approximately: 4.3*10-60.

### Methods for generating random strings, compare hash codes and write final results to csv

In [30]:
def passwords(encoding):
    chars = [c.encode(encoding) for c in printable]
    for length in count(start=1):
        for pwd in product(chars, repeat=length):
            yield b''.join(pwd)
            
            
def crack(hash_function, search_hash, encoding):
    for pwd in passwords(encoding):
        if hash_function(pwd).digest() == search_hash:
            return pwd.decode(encoding)
        
        
def append_list_to_csv(file_name, list_of_elem):
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)
        
        
def calculate_cracking_time(pass_to_crack, encoding, result_file):
    hash_funcs = (md5, sha256, sha512)
    hash_funcs_names = ('MD5', 'SHA256', 'SHA512')
    
    for i in range(3):
        hashed_pass = hash_funcs[i](pass_to_crack.encode(encoding)).digest()
        
        start = time()
        cracked = crack(hash_funcs[i], hashed_pass, encoding)
        end = time()
        duration = round(end - start, 6)
        
        print(f"Password cracked with {hash_funcs_names[i]}: {cracked} for {duration} seconds")
        append_list_to_csv(result_file,[hash_funcs_names[i], pass_to_crack, cracked, duration])  

    

### Initial data and passwords hash generation

In [31]:
encoding = 'ascii'  # utf-8 for unicode support
password = 'rev'
csv_file = 'data.csv'

calculate_cracking_time(password, encoding, csv_file)


Password cracked with MD5: rev for 0.271283 seconds
Password cracked with SHA256: rev for 0.32682 seconds
Password cracked with SHA512: rev for 0.34587 seconds


### For given password calculate time and write output to file

b'K\xe9\xb5\xf8E\x0f\xadgg\x19<\x7ff\x9a\xd9\xc6\xb3\xb5\xe8\x9c\xafac(M\x0b\xf5\xd2\xd9\xc0\xa7\xfb\x04\x94f\xe7\xf4\x98\x1f\xbf\xaaB\x14\xaf\x82\x8e\xde\\\xf6s\xb8\xaet\xe5\xa9\xe8\x01\xc6\xa6\x1e\x1d\xdc\xb8\x11'
